# Piezo Crystal Voltage Responce

In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mp
import matplotlib.pyplot as plt 
import uncertainties as unc
import scipy as sci

from uncertainties import ufloat
from scipy.optimize import curve_fit

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
# Function Generator Voltage
FV = pd.read_csv('Data/F0000CH1.CSV', header=None, index_col=False, usecols=[3, 4])

# Piezo Voltage
PV = pd.read_csv('Data/F0000CH4.CSV', header=None, index_col=False, usecols=[3, 4])

# Photodetector Voltage 
DV = pd.read_csv('Data/F0000CH2.CSV', header=None, index_col=False, usecols=[3, 4])

In [3]:
# FV.rename(columns={'3':'Time (s)', '4':'Voltage (s)'}, inplace=True)
FV.columns = ['Time (s)', 'Voltage (V)']
PV.columns = ['Time (s)', 'Voltage (mV)']
DV.columns = ['Time (s)', 'Voltage (V)']

## Plot the Data

In [4]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

FV.plot(x = 'Time (s)', y = 'Voltage (V)' , ax = axes[0], legend=False,
        title = 'Voltage output of function generator',
        xlabel = 's',
        ylabel = 'V')

PV.plot(x = 'Time (s)', y = 'Voltage (mV)', ax = axes[1], legend=False,
        title = 'Voltage applied to piezo crystal',
        xlabel = 's',
        ylabel = 'mV')

DV.plot(x = 'Time (s)', y = 'Voltage (V)', ax = axes[2], legend=False,
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V')

axes[0].grid()
axes[1].grid()
axes[2].grid()
plt.tight_layout()
plt.show()

<Figure size 864x216 with 3 Axes>

## Curve fit the voltage output of photodetector

In [5]:
# Function
def DVFunc(x, a, b, c, d, a2, b2, c2):
    f = a2*np.sin(np.pi*b2*x + c2*np.pi)
    F = a*np.sin(np.pi*(b + f)*x + c*np.pi) + d
    return F

def LinFunc(x, a, b):
    return a*x + b

In [6]:
# Find positions of min and max values of PV
min_pos, max_pos = np.argmin(PV['Voltage (mV)'])+100, np.argmax(PV['Voltage (mV)'])

In [7]:
# Constrain DV and PV elements to those between min and max of PV
DVTime, DVVolt = DV['Time (s)'][min_pos:max_pos], DV['Voltage (V)'][min_pos:max_pos]
PVTime, PVVolt = PV['Time (s)'][min_pos:max_pos], PV['Voltage (mV)'][min_pos:max_pos]

# Curve Fit
DVpopt, DVpcov = curve_fit(DVFunc, DVTime, DVVolt, 
                       bounds = ([0.15, 1000, -1, -1, 0, 0, -1], 
                                 [1, np.inf, 1, 0, np.inf, np.inf, 1]))

PVpopt, PVpcov = curve_fit(LinFunc, PVTime, PVVolt)

a, b, c, d, a2, b2, c2 = DVpopt

# Normalize photodetector intensity
DV['Intensity'] = (DV['Voltage (V)'] - np.min(DV['Voltage (V)'])) / (np.max(DV['Voltage (V)']) - np.min(DV['Voltage (V)']))

DVIntn = DV['Intensity'][min_pos:max_pos]

DVIpopt, DVIpcov = curve_fit(DVFunc, DVTime, DVIntn, 
                             bounds = ([0.45, 1000, -1, 0.4, 0, 0, -1], 
                                       [0.55, np.inf, 1, 0.6, np.inf, np.inf, 1]))

aI, bI, cI, dI, aI2, bI2, cI2 = DVIpopt

In [8]:
# Plot it out 
fig, ax = plt.subplots(3, 1, figsize = (8, 12))

# Voltage applied to piezo  per time

PV.plot(x = 'Time (s)', y = 'Voltage (mV)', ax = ax[0], legend=False,
        title = 'Voltage applied to piezo crystal',
        xlabel = 's',
        ylabel = 'mV')

ax[0].plot(PVTime, LinFunc(PVTime, *PVpopt),
           label = r'Fit: $y = ({0:.2g}) t  {1:.2g}$'.format(PVpopt[0], PVpopt[1]))

ax[0].legend()
ax[0].grid()

# Voltage output of photodetector per time
DV.plot(x = 'Time (s)', y = 'Voltage (V)', legend=False, ax = ax[1],
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V',
        label = 'Data')

ax[1].plot(DVTime, DVFunc(DVTime, *DVpopt), 
          label = r'Fit: $y = {0:.2g} \sin[\nu \pi t {2:.2g} \pi] + {3:.2g}$' '\n' 
           r'Where $\nu= {1:.2g} + {4:.2g} \sin({5:.2g} \pi t + {6:.2g}\pi)$'.format(a, b, c, d, a2, b2, c2))

ax[1].set_ylim(np.min(DV['Voltage (V)'])*1.3)
ax[1].legend(loc='lower left')
ax[1].grid()

# Normalized intensity of photodetector with time
DV.plot(x = 'Time (s)', y = 'Intensity', legend=False, ax = ax[2],
        title = 'Normalized output of photodetector',
        xlabel = 's',
        ylabel = 'V',
        label = 'Data')

ax[2].plot(DVTime, DVFunc(DVTime, *DVIpopt), 
          label = r'Fit: $y = {0:.2g} \sin[\nu \pi t {2:.2g} \pi] + {3:.2g}$' '\n' 
           r'Where $\nu= {1:.2g} + {4:.2g} \sin({5:.2g} \pi t + {6:.2g}\pi)$'.format(aI, bI, cI, dI, aI2, bI2, cI2))

ax[2].set_ylim(np.min(DV['Intensity']) - 0.35)
ax[2].legend(loc='lower left')
ax[2].grid()

plt.tight_layout()
plt.show()

<Figure size 576x864 with 3 Axes>

## Displacement of Piezo vs Voltage

In [9]:
# Plot piezo displacement vs Voltage
IntNormDat = DVIntn
IntNormFit = DVFunc(DVTime, *DVIpopt)

def IntToDispFunc(x, a=aI, b=bI, c=cI, d=dI, lam=632, f=2):
    A = (x-d)/a
    Num = np.arcsin(A) - c*np.pi
    Den = b*np.pi 
    F = Num/Den
    return F*lam/f

# Plot it out

plt.plot(IntToDispFunc(DVIntn), PVVolt)

plt.xlabel('Displacement ($\lambda$)')
plt.ylabel('Voltage (V)')
plt.title('Piezo displacement per applied voltage')
plt.grid()
plt.show()

/Users/kev/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)


<Figure size 432x288 with 1 Axes>

### Graphing the voltage output of the photodetector against voltage applied to 

In [10]:

plt.plot(PV['Voltage (mV)'], DV['Voltage (V)'])



<Figure size 432x288 with 1 Axes>

## Figuring out the displacement of piezo crystal per volt

Wavelength of the laser: $\lambda = 632$ nm

Frequency of the photodetector: $\nu_D = 3969.668$ Hz or $\lambda / s$

This means that the piezo is moving at $v_P = \nu_D / 2$ $\lambda/s$

For that period, the voltage applied to the piezo is increasing at $1.7 \times 10^4$ mV/s = $17$ V/s

